In [23]:
#Import Statements
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


AttributeError: module 'tensorflow' has no attribute 'name_scope'

In [8]:
train_reviews = []
test_reviews = []

for line in open('full_test.txt', 'r', encoding = 'utf8'):
    test_reviews.append(line.strip())

for line in open('full_train.txt', 'r', encoding = 'utf8'):
    train_reviews.append(line.strip())

In [9]:
#Preprocessing the data
SPECIAL_CHARACTERS = re.compile("[.;:!\'?,\"()\[\]]")
HTML_CHARACTERS = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

In [10]:
def preprocess_data(reviews):
  reviews = [SPECIAL_CHARACTERS.sub("", line.lower()) for line in reviews]
  reviews = [HTML_CHARACTERS.sub(" ", line) for line in reviews]
  return reviews

In [11]:
processed_train_reviews = preprocess_data(train_reviews)
processed_test_reviews = preprocess_data(test_reviews)

In [12]:
#Removing Stopwords
stopwords_english = stopwords.words('english')
def remove_stop_words(data):
  removed_words = []
  for review in data:
    removed_words.append(' '.join([word for word in review.split() if word not in stopwords_english]))
  return removed_words

In [13]:
no_stop_words_train = remove_stop_words(processed_train_reviews)
no_stop_words_test = remove_stop_words(processed_test_reviews)

In [14]:
#Normalization

#Stemming
def get_stemmed_text(corpus):
  stemmer = PorterStemmer()
  return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

#Lemmatization
def get_lemmatized_text(corpus):
  lemmatizer = WordNetLemmatizer()
  return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]


In [15]:
train_reviews = get_stemmed_text(no_stop_words_train)
train_reviews = get_lemmatized_text(train_reviews)
test_reviews = get_stemmed_text(no_stop_words_test)
test_reviews = get_lemmatized_text(test_reviews)

In [17]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(train_reviews)
X = ngram_vectorizer.transform(train_reviews)

In [20]:
X_test = ngram_vectorizer.transform(test_reviews)
X_train = sequence.pad_sequences(X_train, maxlen=500)
X_test = sequence.pad_sequences(X_test, maxlen=500)
target = [1 if i < 12500 else 0 for i in range(25000)]
X_train, X_val, y_train, y_val = train_test_split(X,target)

NameError: name 'sequence' is not defined

In [19]:
X_train.size
y_train.size

AttributeError: 'list' object has no attribute 'size'

In [0]:
model = tf.keras.Sequential()
model.add(layers.Embedding(5000, 32, input_length=500))
model.add(layers.Flatten())
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 16000)             0         
_________________________________________________________________
dense (Dense)                (None, 250)               4000250   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 251       
Total params: 4,160,501
Trainable params: 4,160,501
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
#model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=128, verbose=2)
model.fit(x =X_train, y=y_train ,epochs=10,batch_size = 128, validation_data=(X_val, y_val))
scores = model.evaluate(X_val, y_val, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

ValueError: ignored